In [1]:
# Import basic libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import twitter library
import tweepy as tw

# Import json library for authentication
import json

In [2]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [3]:
# Authenticate to Twitter
# The Twitter API uses OAuth, a widely used open authorization protocol, to authenticate all the requests.

auth = tw.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

# Create a new tweepy.API object
# The API class has many methods that provide access to Twitter API endpoints. 
# Using these methods, you can access the Twitter API’s functionality

api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
# Verify the credentials
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [5]:
# Post a tweet from Python

api.update_status("First tweet from python notebook!")

Status(_api=<tweepy.api.API object at 0x0000028E0D2261C0>, _json={'created_at': 'Thu Feb 18 04:13:11 +0000 2021', 'id': 1362253804180242436, 'id_str': '1362253804180242436', 'text': 'First tweet from python notebook!', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1175413689333297152, 'id_str': '1175413689333297152', 'name': 'Amit Saini', 'screen_name': 'MrAmitSaini', 'location': 'Toronto, Canada', 'description': 'Data Science Enthusiast!!!', 'url': 'https://t.co/lsfZHkU24T', 'entities': {'url': {'urls': [{'url': 'https://t.co/lsfZHkU24T', 'expanded_url': 'https://www.linkedin.com/in/mr-amit-saini/', 'display_url': 'linkedin.com/in/mr-amit-sai…', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 4, 'friends

In [6]:
# To get trending hashtags from twitter use trends_place=1 means worlwide
# The id parameter for this endpoint is the "where on earth identifier" or WOEID
# WOEID locations include: Worldwide: 1 Canada: 23424775 United States: 23424977
# You can see the full list of available locations using api.trends_available().

toronto_trend_result = api.trends_place(4118)
toronto_trend_result

[{'trends': [{'name': 'Sens',
    'url': 'http://twitter.com/search?q=Sens',
    'promoted_content': None,
    'query': 'Sens',
    'tweet_volume': 38079},
   {'name': 'Holl',
    'url': 'http://twitter.com/search?q=Holl',
    'promoted_content': None,
    'query': 'Holl',
    'tweet_volume': None},
   {'name': '#coronercbc',
    'url': 'http://twitter.com/search?q=%23coronercbc',
    'promoted_content': None,
    'query': '%23coronercbc',
    'tweet_volume': None},
   {'name': 'Freddie',
    'url': 'http://twitter.com/search?q=Freddie',
    'promoted_content': None,
    'query': 'Freddie',
    'tweet_volume': None},
   {'name': 'Engvall',
    'url': 'http://twitter.com/search?q=Engvall',
    'promoted_content': None,
    'query': 'Engvall',
    'tweet_volume': None},
   {'name': 'Horwath',
    'url': 'http://twitter.com/search?q=Horwath',
    'promoted_content': None,
    'query': 'Horwath',
    'tweet_volume': None},
   {'name': 'Andrea',
    'url': 'http://twitter.com/search?q=Andre

In [7]:
for trend in toronto_trend_result[0]["trends"]:
    print(trend["name"])

Sens
Holl
#coronercbc
Freddie
Engvall
Horwath
Andrea
steve simmons
CERB
#TexasBlackout
Willy
shawn
Roman
Serena
Naomi
#AEWDynamite
McDavid
#RHOSLC
#RHONJ
Puljujarvi
Copp
Jordie Benn
Splatoon 3
Matthews
Tatis
Meek
Ehlers
Draisaitl
Skyward Sword
Joe Thornton
Rielly
Stepan
Jamal Murray
hutch
Rush
Gudbranson
AIDS
Riho
Mario Golf
Teresa
Pyra
Jacob Markstrom
Simon
Anisimov
Nintendo
Marner
Lolo
Sting
Kerfoot
Juolevi


In [8]:
# Get the 10 most recent public tweets that are in English and contain the word "Python"

for tweet in api.search(q="Python", lang="en", rpp=10):
    print(f"{tweet.user.name}: {tweet.text}")

Amit Saini: First tweet from python notebook!
EnterpriseApplications.org - the Bot: RT @Password_Gen: Secure and random passphrase: cupules stoops hornlike muskies | Strength: 0.951577/1.0 | #passphrase #passwords #security…
Pythonista Bot: RT @Password_Gen: Secure and random passphrase: cupules stoops hornlike muskies | Strength: 0.951577/1.0 | #passphrase #passwords #security…
Django Bot: RT @Password_Gen: Secure and random passphrase: cupules stoops hornlike muskies | Strength: 0.951577/1.0 | #passphrase #passwords #security…
joe coope: @CharlesSzulc Remember the Monty python script "Aussies in London pub" all the bloke's are called...
SecurePassphrases: Secure and random passphrase: cupules stoops hornlike muskies | Strength: 0.951577/1.0 | #passphrase #passwords… https://t.co/8VWlt7MaWr
Code Newbie Bot: RT @byLilyV: #FEATURED #COURSES
Complete #Python #Developer in 2021: Zero to Mastery
How to become a #Python3 Developer and get hired
Build…
DQServ: RT @GavLaaaaaaaa: WebGPU is com

In [17]:
# Streaming allows you to actively watch for tweets that match certain criteria in real time. 
# This means that when there aren’t any new tweet matching the criteria, then the program will
# wait until a new tweet is created and then process it. To use streaming you have to create two objects:
# 1) The stream object uses the Twitter API to get tweets that match some criteria. This object is the source of tweets that are then processed by a stream listener.
# 2) The stream listener receives tweets from the stream.


class MyStreamListener(tw.StreamListener):
    def __init__(self, api):
        self.api = api
        self.me = api.me()
        
    # Tweets from the stream are processed by on_status().    
    def on_status(self, tweet):
        print(f"{tweet.user.name}:{tweet.text}")

    def on_error(self, status):
        print("Error detected")

tweets_listener = MyStreamListener(api)
stream = tw.Stream(api.auth, tweets_listener)

# To start getting tweets from the stream, you have to call the stream’s filter(),passing the criteria to use to filter tweets. 
# Then, for each new tweet that matches the criteria, the stream object invokes the stream listener’s on_status().
stream.filter(track=["Python", "Django", "Tweepy"], languages=["en"])

uh-oh:RT @gp_pulipaka: Essential Math for Data Science. #BigData #Analytics #DataScience #AI #MachineLearning #IoT #IIoT #Python #RStats #TensorF…
Monster Coder:Unqiue letter processing question that will tease your brain #lintcode #python 

https://t.co/zTSkGOlpMc
Vikram_Singh Rawat:RT @deconstructized: @WeAreRLadies I am in the Python part of the R community and I don't intend that antagonistically.  Your language comm…
uh-oh:RT @Sirlupinwatson: What is Machine Learning? 

https://t.co/37c06GRCFr

#MachineLearning #Python #NLP #Analytics #AI #100DaysOfCode
#DEVCo…
uh-oh:RT @Sirlupinwatson: How To Learn Statistics For Data Science, The Self-Starter Way

https://t.co/t3Vse0xm7q

#MachineLearning #Python #NLP…
Pythonista Bot:RT @mipsmonsta: Unqiue letter processing question that will tease your brain #lintcode #python 

https://t.co/zTSkGOlpMc
Django Bot:RT @mipsmonsta: Unqiue letter processing question that will tease your brain #lintcode #python 

https://t.co/zTSkGOlpMc
Purevdorj Enk

KeyboardInterrupt: 

In [10]:
# Another method to use twitter API
import twitter

# Authenticate access to twitter app
auth=twitter.oauth.OAuth(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'], creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
twitter_api=twitter.Twitter(auth=auth)

In [11]:
world_trend_result=twitter_api.trends.place(_id=1)
print (world_trend_result)

[{'trends': [{'name': 'スプラ3', 'url': 'http://twitter.com/search?q=%E3%82%B9%E3%83%97%E3%83%A93', 'promoted_content': None, 'query': '%E3%82%B9%E3%83%97%E3%83%A93', 'tweet_volume': 175313}, {'name': 'Serena', 'url': 'http://twitter.com/search?q=Serena', 'promoted_content': None, 'query': 'Serena', 'tweet_volume': 65898}, {'name': 'Switch', 'url': 'http://twitter.com/search?q=Switch', 'promoted_content': None, 'query': 'Switch', 'tweet_volume': 544618}, {'name': 'スプラトゥーン3', 'url': 'http://twitter.com/search?q=%E3%82%B9%E3%83%97%E3%83%A9%E3%83%88%E3%82%A5%E3%83%BC%E3%83%B33', 'promoted_content': None, 'query': '%E3%82%B9%E3%83%97%E3%83%A9%E3%83%88%E3%82%A5%E3%83%BC%E3%83%B33', 'tweet_volume': 220626}, {'name': '#RHOSLC', 'url': 'http://twitter.com/search?q=%23RHOSLC', 'promoted_content': None, 'query': '%23RHOSLC', 'tweet_volume': None}, {'name': 'Meek', 'url': 'http://twitter.com/search?q=Meek', 'promoted_content': None, 'query': 'Meek', 'tweet_volume': 21833}, {'name': '#RHONJ', 'url': 

In [12]:
for trend in world_trend_result[0]["trends"]:
    print(trend["name"])

スプラ3
Serena
Switch
スプラトゥーン3
#RHOSLC
Meek
#RHONJ
#AEWDynamite
#MAFS
#Merciถ้าไบร์ทรับจะรักป่ะ
Splatoon 3
Tatis
YA ES TIEMPO
Rush
Padres
スカウォ
Paige
Skyward Sword
戦国無双5
Nintendo
LINEMO
コミカライズ
スマブラ
スパショ
聖剣LOM
Mario Golf
リマスター
BELLE OnMagandangBuhay
しんちゃん
Tim Tebow
Zelda
ゼノブレ2
モンハン
Pyra
DepEd
スカイウォードソード
ヤンキー片想い中
Bulls
会長受諾の意向
フォールガイズ
ゼノブレイド2
Teresa
ホムラとヒカリ
Mythra
ニンダイ
AIDS
Marco Pérez
Cruz Azul
コジャケ
Jerami Grant


In [13]:
# Search for the q, only 100 tweets will be returned. 
# The maximum number of tweets that can be searched is up to 18,000 tweets and within last 9 days

search_term = "lockdown"
search_results=twitter_api.search.tweets(q=search_term,count=100)

# A status is a tweet.
statuses = search_results ['statuses']

In [14]:
# See twitter data in a table format
tweetframes=pd.DataFrame(statuses)

In [15]:
tweetframes.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,quoted_status_id,quoted_status_id_str,extended_entities
0,Thu Feb 18 04:13:13 +0000 2021,1362253815114731520,1362253815114731520,RT @theage: A team of Australian researchers i...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,False,4,0,False,False,en,NaN,NaN,NaN,NaN
1,Thu Feb 18 04:13:13 +0000 2021,1362253814208729094,1362253814208729094,RT @SantoshRanjan_: Year LPG Price\n2011 ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'und', 'result_type': 'r...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,False,693,0,False,False,und,NaN,NaN,NaN,NaN
2,Thu Feb 18 04:13:13 +0000 2021,1362253813441265665,1362253813441265665,RT @RolfatWarwick: Rolf report 18 Feb: A simpl...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,False,3,0,False,False,en,NaN,NaN,NaN,NaN
3,Thu Feb 18 04:13:12 +0000 2021,1362253808714215427,1362253808714215427,@VincentRK I mean I really don't understand wh...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.362253e+18,1362252702378418178,...,False,0,0,False,False,en,NaN,NaN,NaN,NaN
4,Thu Feb 18 04:13:11 +0000 2021,1362253808215097345,1362253808215097345,Year LPG Price\n2011 ₹ 877\n2012 ...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,False,0,0,False,False,en,NaN,NaN,NaN,NaN


In [16]:
tweetframes.shape

(100, 29)